# Análise e Previsão com Prophet
## Inventory Anomaly Detector

Este notebook realiza o treinamento e análise de modelos Prophet para previsão de consumo.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Adicionar src ao path
sys.path.insert(0, str(Path.cwd()))

from src.data_loader import load_raw_consumo, load_inventory_data
from src.data_cleaning import clean_consumo
from src.forecasting import (
    train_prophet_model,
    make_forecast,
    forecast_7_days,
    train_models_by_product,
    forecast_7_days_by_product,
    evaluate_model,
    save_model,
    load_model
)
from src.config import PRODUCT_COLUMN, DATE_COLUMN

print("Bibliotecas importadas com sucesso!")


## 1. Carregar e Preparar Dados


In [ ]:
# Carregar dados de consumo
consumo_df = load_raw_consumo()

print(f"\nDados carregados:")
print(f"  - Total de registros: {len(consumo_df)}")
print(f"  - Produtos: {consumo_df[PRODUCT_COLUMN].nunique() if PRODUCT_COLUMN in consumo_df.columns else 'N/A'}")
print(f"\nPrimeiras linhas:")
consumo_df.head()


In [ ]:
# Limpar dados de consumo
consumo_limpo = clean_consumo(
    consumo_df,
    remove_outliers=True,
    fill_missing=True,
    fill_method="interpolate"
)

print(f"\nDados limpos: {len(consumo_limpo)} registros")
consumo_limpo.head()


## 2. Treinar Modelo Prophet para um Produto


In [ ]:
# Selecionar um produto para análise detalhada
if PRODUCT_COLUMN in consumo_limpo.columns:
    produtos = consumo_limpo[PRODUCT_COLUMN].unique()
    produto_selecionado = produtos[0]
    print(f"Produto selecionado: {produto_selecionado}")
    
    # Filtrar dados do produto
    df_produto = consumo_limpo[consumo_limpo[PRODUCT_COLUMN] == produto_selecionado].copy()
    df_produto = df_produto[["ds", "y"]].sort_values("ds").reset_index(drop=True)
else:
    # Se não houver produto_id, usar todos os dados
    df_produto = consumo_limpo[["ds", "y"]].sort_values("ds").reset_index(drop=True)
    produto_selecionado = "TODOS"

print(f"\nDados do produto:")
print(f"  - Registros: {len(df_produto)}")
print(f"  - Período: {df_produto['ds'].min()} a {df_produto['ds'].max()}")
df_produto.head()


In [ ]:
# Treinar modelo Prophet
model = train_prophet_model(df_produto, product_id=produto_selecionado)


## 3. Visualizar Componentes do Modelo


In [ ]:
# Fazer previsão incluindo histórico
forecast = model.predict(model.make_future_dataframe(periods=0))

# Plotar componentes do modelo
fig = model.plot_components(forecast, figsize=(15, 10))
plt.tight_layout()
plt.show()


## 4. Visualizar Previsão vs Dados Históricos


In [ ]:
# Plotar previsão com dados históricos
fig = model.plot(forecast, figsize=(15, 6))
plt.title(f'Previsão Prophet - {produto_selecionado}')
plt.xlabel('Data')
plt.ylabel('Consumo')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 5. Previsão para os Próximos 7 Dias


In [ ]:
# Fazer previsão de 7 dias
forecast_7d = forecast_7_days(model, include_history=False)

print("Previsão para os próximos 7 dias:")
print(forecast_7d.to_string(index=False))


In [ ]:
# Visualizar previsão de 7 dias
fig, ax = plt.subplots(figsize=(15, 6))

# Dados históricos (últimos 30 dias)
df_recente = df_produto.tail(30)
ax.plot(df_recente['ds'], df_recente['y'], 'o-', label='Dados Históricos', linewidth=2, markersize=4)

# Previsão de 7 dias
ax.plot(forecast_7d['ds'], forecast_7d['yhat'], 's-', label='Previsão', linewidth=2, markersize=6, color='red')
ax.fill_between(forecast_7d['ds'], forecast_7d['yhat_lower'], forecast_7d['yhat_upper'], 
                alpha=0.3, color='red', label='Intervalo de Confiança')

ax.set_title(f'Previsão de 7 Dias - {produto_selecionado}', fontsize=14, fontweight='bold')
ax.set_xlabel('Data')
ax.set_ylabel('Consumo')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 6. Treinar Modelos para Todos os Produtos


In [ ]:
# Preparar dados com produto_id
if PRODUCT_COLUMN not in consumo_limpo.columns:
    # Adicionar produto_id se não existir
    df_completo = load_inventory_data()
    consumo_limpo = consumo_limpo.merge(
        df_completo[[DATE_COLUMN, PRODUCT_COLUMN]].drop_duplicates(),
        left_on="ds",
        right_on=DATE_COLUMN,
        how="left"
    )
    consumo_limpo = consumo_limpo.drop(columns=[DATE_COLUMN])
    consumo_limpo = consumo_limpo.dropna(subset=["produto_id"])

# Treinar modelos para todos os produtos
prophet_models = train_models_by_product(
    consumo_limpo,
    product_column="produto_id",
    save_models=True
)

print(f"\nModelos treinados: {len(prophet_models)}")
print(f"Produtos: {list(prophet_models.keys())}")


## 7. Previsão de 7 Dias para Todos os Produtos


In [ ]:
# Gerar previsões de 7 dias para todos os produtos
forecast_all_7d = forecast_7_days_by_product(prophet_models)

print(f"\nPrevisões geradas: {len(forecast_all_7d)} registros")
print(f"\nPrimeiras previsões:")
forecast_all_7d.head(10)


In [ ]:
# Visualizar previsões de todos os produtos
n_produtos = forecast_all_7d['produto_id'].nunique()
fig, axes = plt.subplots(n_produtos, 1, figsize=(15, 4*n_produtos))

if n_produtos == 1:
    axes = [axes]

for idx, produto in enumerate(forecast_all_7d['produto_id'].unique()):
    # Dados históricos do produto
    df_prod = consumo_limpo[consumo_limpo['produto_id'] == produto].tail(30)
    
    # Previsões do produto
    forecast_prod = forecast_all_7d[forecast_all_7d['produto_id'] == produto]
    
    axes[idx].plot(df_prod['ds'], df_prod['y'], 'o-', label='Histórico', linewidth=2, markersize=3)
    axes[idx].plot(forecast_prod['ds'], forecast_prod['yhat'], 's-', label='Previsão', 
                   linewidth=2, markersize=5, color='red')
    axes[idx].fill_between(forecast_prod['ds'], forecast_prod['yhat_lower'], forecast_prod['yhat_upper'],
                           alpha=0.3, color='red', label='Intervalo de Confiança')
    
    axes[idx].set_title(f'Previsão de 7 Dias - {produto}', fontweight='bold')
    axes[idx].set_xlabel('Data')
    axes[idx].set_ylabel('Consumo')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 8. Análise de Previsões por Produto


In [ ]:
# Estatísticas das previsões por produto
print("Estatísticas das Previsões de 7 Dias por Produto:")
print("=" * 60)

for produto in forecast_all_7d['produto_id'].unique():
    forecast_prod = forecast_all_7d[forecast_all_7d['produto_id'] == produto]
    
    print(f"\n{produto}:")
    print(f"  - Consumo médio previsto: {forecast_prod['yhat'].mean():.2f}")
    print(f"  - Consumo mínimo previsto: {forecast_prod['yhat'].min():.2f}")
    print(f"  - Consumo máximo previsto: {forecast_prod['yhat'].max():.2f}")
    print(f"  - Amplitude do intervalo: {forecast_prod['yhat_upper'].mean() - forecast_prod['yhat_lower'].mean():.2f}")
    
    # Comparar com histórico recente
    if PRODUCT_COLUMN in consumo_limpo.columns:
        historico = consumo_limpo[consumo_limpo['produto_id'] == produto].tail(7)
        if len(historico) > 0:
            consumo_medio_historico = historico['y'].mean()
            print(f"  - Consumo médio histórico (últimos 7 dias): {consumo_medio_historico:.2f}")
            variacao = ((forecast_prod['yhat'].mean() - consumo_medio_historico) / consumo_medio_historico) * 100
            print(f"  - Variação prevista: {variacao:+.2f}%")


## 9. Salvar Previsões


In [ ]:
# Salvar previsões de 7 dias
from src.data_cleaning import save_processed

output_path = Path("outputs/forecast_7d_prophet.csv")
save_processed(forecast_all_7d, output_path, format="csv")

print(f"Previsões salvas em: {output_path}")
print(f"Total de registros: {len(forecast_all_7d)}")


## 10. Resumo e Conclusões


In [ ]:
print("=" * 60)
print("RESUMO DA ANÁLISE COM PROPHET")
print("=" * 60)
print(f"\n1. Modelos treinados: {len(prophet_models)}")
print(f"2. Produtos analisados: {list(prophet_models.keys())}")
print(f"3. Previsões geradas: {len(forecast_all_7d)} registros (7 dias × {len(prophet_models)} produtos)")
print(f"4. Período de previsão: {forecast_all_7d['ds'].min()} a {forecast_all_7d['ds'].max()}")

print("\n=== PRÓXIMOS PASSOS ===")
print("- Usar previsões para detectar anomalias")
print("- Aplicar Isolation Forest nos resíduos")
print("- Comparar previsões com valores reais quando disponíveis")
print("- Ajustar hiperparâmetros do Prophet se necessário")
